In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (1000 Biscayne Blvd #57, Miami, FL 33132, USA,...
2     (1300 S Miami Ave, Miami, FL 33130, USA, (25.7...
3     (16901 Collins Ave #3705, Sunny Isles Beach, F...
4     (4775 Collins Ave APT 4003, Miami Beach, FL 33...
5     (5000 Island Estates Dr #1007, Aventura, FL 33...
6     (101 20th St #2201, Miami Beach, FL 33139, USA...
7     (19123 Fisher Island Dr, Miami Beach, FL 33109...
8     (400 Alton Rd #4a, Miami Beach, FL 33139, USA,...
9     (Sunny Isles Beach, FL 33160, USA, (25.9253941...
10    (68 SE 6th St Apt 2603, Miami, FL 33131, USA, ...
Name: loc, dtype: object

In [9]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [10]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,One Thousand Museum\r1000 Biscayne Blvd PH-57\...,5/27/2022,9/29/2022,116,"$19,500,000.00",9200,"$2,119.57",Julian Johnston,The Corcoran Group,Christopher Wands,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd PH-57 M...,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami,"(1000 Biscayne Blvd #57, Miami, FL 33132, USA,...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
2,SLS Brickell\r1300 S Miami Ave UPH3\rMiami,6/28/2022,9/29/2022,63,"$4,600,000.00",2268,"$2,028.22",MariaClaudia Fernandez,One Sotheby's Int'l Realty,David Freed,Keller Williams Miami Beach Realty,SLS Brickell 1300 S Miami Ave UPH3 Miami,SLS Brickell,1300 S Miami Ave UPH3 Miami,"(1300 S Miami Ave, Miami, FL 33130, USA, (25.7...","(25.7612047, -80.19359659999999, 0.0)",25.761205,-80.193597,0.0
3,Jade Signature\r16901 Collins Ave 3705\rSunny ...,12/8/2021,9/28/2022,226,"$4,500,000.00",3405,"$1,321.59",Marina Osmacova,MIAMI VIP REALTY LLC,Steven Seigel,"Compass Florida, LLC",Jade Signature 16901 Collins Ave 3705 Sunny Is...,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach,"(16901 Collins Ave #3705, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
4,Green Diamond\r4775 Collins Ave 4003\rMiami Beach,4/26/2022,9/30/2022,58,"$3,550,000.00",1980,"$1,792.93",Claudia Whitechurch,Coldwell Banker Realty,Menachem Fellig,"Compass Florida, LLC",Green Diamond 4775 Collins Ave 4003 Miami Beach,Green Diamond,4775 Collins Ave 4003 Miami Beach,"(4775 Collins Ave APT 4003, Miami Beach, FL 33...","(25.8227293, -80.1226745, 0.0)",25.822729,-80.122675,0.0
5,Prive\r5000 Island Estates Dr 1007\rAventura,6/30/2022,9/30/2022,29,"$3,500,000.00",2830,"$1,236.75",Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Prive 5000 Island Estates Dr 1007 Aventura,Prive,5000 Island Estates Dr 1007 Aventura,"(5000 Island Estates Dr #1007, Aventura, FL 33...","(25.9470842, -80.1306458, 0.0)",25.947084,-80.130646,0.0
6,The Setai\r101 20th St 2201\rMiami Beach,6/26/2022,9/27/2022,31,"$3,300,000.00",1198,"$2,754.59",Cecilia Solorzano,Suma Luxury Realty,Matt Lill,The Corcoran Group,The Setai 101 20th St 2201 Miami Beach,The Setai,101 20th St 2201 Miami Beach,"(101 20th St #2201, Miami Beach, FL 33139, USA...","(25.7956082, -80.1281603, 0.0)",25.795608,-80.128160,0.0
7,Fisher Island\r19123 Fisher Island Dr 19123\rM...,4/19/2022,9/27/2022,118,"$2,430,000.00",1875,"$1,296.00",Nataly Stuart,"Socialite Realty, LLC",Gregory Gringruz,The Island,Fisher Island 19123 Fisher Island Dr 19123 Mia...,Fisher Island,19123 Fisher Island Dr 19123 Miami Beach,"(19123 Fisher Island Dr, Miami Beach, FL 33109...","(25.7577288, -80.1386049, 0.0)",25.757729,-80.138605,0.0
8,Murano Grande\r400 Alton Rd TH-4A\rMiami Beach,4/12/2022,9/30/2022,127,"$1,920,000.00",1912,"$1,004.18",Barbara Smith,"Golden Realty Group, Inc.",Oliver Davis,Keller Williams Miami Beach Realty,Murano Grande 400 Alton Rd TH-4A Miami Beach,Murano Grande,400 Alton Rd TH-4A Miami Beach,"(400 Alton Rd #4a, Miami Beach, FL 33139, USA,...","(25.7732967, -80.1403323, 0.0)",25.773297,-80.140332,0.0
9,St Tropez on the Bay\r150 Sunny Isle Blvd 1-UP...,4/5/2022,9/27/2022,76,"$1,700,000.00",3020,$562.91,Bento Queiroz,"Compass Florida, LLC",Gina Nash,Aston Rose Sports And Entertai,St Tropez on the Bay 150 Sunny Isle Blvd 1-UPH...,St Tropez on the Bay,150 Sunny Isle Blvd 1-UPH3 Sunny Isles Beach,"(Sunny Isles Beach, FL 33160, USA, (25.9253941...","(25.9253941, -80.12083129999999, 0.0)",25.925394,-80.120831,0.0
10,R0each Condo\r68 SE 6th St 2603\rMiami,11/11/2021,9/27/2022,54,"$1,700,000.00",1891,$899.00,Claudia Lopez,Douglas Elliman,Ross Milroy,The Corcoran Group,R0each Condo 68 SE 6th St 2603 Miami,R,0each Condo 68 SE 6th St 2603 Miami,"(68 SE 6th St Apt 2603

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [22]:
df.at[10,'building_name']=('Reach Condo')
df.at[10,'address_only']=('68 SE 6th St 2603')

df.at[4,'Buyer Agent']=('Mendel Fellig')

df.at[2,'Agent']=('Claudia Fernandez')

In [18]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [19]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('September 25th - October 1st')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [20]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [16]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_100122
